# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
R. Andrassy  ->  R. Andrassy  |  ['R. Andrassy']
J. Liu  ->  J. Liu  |  ['J. Liu']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
P. Molliere  ->  P. Molliere  |  ['P. Molliere']
Arxiv has 80 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.07637


extracting tarball to tmp_2410.07637...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.07841


extracting tarball to tmp_2410.07841...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.08116


extracting tarball to tmp_2410.08116...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/Wasp15b_AMP.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=0.9\columnwidth]{figures/Wasp15b_AMP.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 124 bibliographic references in tmp_2410.08116/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.08116-b31b1b.svg)](https://arxiv.org/abs/2410.08116) | **BOWIE-ALIGN: JWST reveals hints of planetesimal accretion and complex sulphur chemistry in the atmosphere of the misaligned hot Jupiter WASP-15b**  |
|| J. Kirk, et al. -- incl., <mark>E.-M. Ahrer</mark>, <mark>P. Molliere</mark> |
|*Appeared on*| *2024-10-11*|
|*Comments*| *24 pages, 23 figures, 6 tables. Submitted to MNRAS*|
|**Abstract**|            We present a transmission spectrum of the misaligned hot Jupiter WASP-15b from 2.8--5.2 microns observed with JWST's NIRSpec/G395H grating. Our high signal to noise data, which has negligible red noise, reveals significant absorption by H$_2$O ($4.2\sigma$) and CO$_2$ ($8.9\sigma$). From independent data reduction and atmospheric retrieval approaches, we infer that WASP-15b's atmospheric metallicity is super-solar ($\gtrsim 15\times$ solar) and its C/O is consistent with solar, that together imply planetesimal accretion. Our GCM simulations for WASP-15b suggest that the C/O we measure at the limb is likely representative of the entire photosphere due to the mostly uniform spatial distribution of H$_2$O, CO$_2$ and CO. We additionally see evidence for absorption by SO$_2$ and absorption at 4.9$\mu$m, for which the current leading candidate is OCS, albeit with several caveats. If confirmed, this would be the first detection of OCS in an exoplanet atmosphere and point towards complex photochemistry of sulphur-bearing species in the upper atmosphere. These are the first observations from the BOWIE-ALIGN survey which is using JWST's NIRSpec/G395H instrument to compare the atmospheric compositions of aligned/low-obliquity and misaligned/high-obliquity hot Jupiters around F stars above the Kraft break. The goal of our survey is to determine whether the atmospheric composition differs across two populations of planets that have likely undergone different migration histories (disc versus disc-free) as evidenced by their obliquities (aligned versus misaligned).         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.07841-b31b1b.svg)](https://arxiv.org/abs/2410.07841) | **From spherical stars to disk-like structures: 3D common-envelope evolution of massive binaries beyond inspiral**  |
|| M. Vetter, et al. -- incl., <mark>R. Andrassy</mark> |
|*Appeared on*| *2024-10-11*|
|*Comments*| *17 Pages, 9 Figures, accepted by A&A*|
|**Abstract**|            Three-dimensional simulations usually fail to cover the entire dynamical common-envelope phase of gravitational wave progenitor systems due to the vast range of spatial and temporal scales involved. We investigated the common-envelope interactions of a $10\,M_\odot$ red supergiant primary star with a black hole and a neutron star companion, respectively, until full envelope ejection (${\gtrsim}\,97 \,\mathrm{\%}$ of the envelope mass). We find that the dynamical plunge-in of the systems determines largely the orbital separations of the core binary system, while the envelope ejection by recombination acts only at later stages of the evolution and fails to harden the core binaries down to orbital frequencies where they qualify as progenitors of gravitational-wave-emitting double-compact object mergers. As opposed to the conventional picture of a spherically symmetric envelope ejection, our simulations show a new mechanism: The rapid plunge-in of the companion transforms the spherical morphology of the giant primary star into a disk-like structure. During this process, magnetic fields are amplified, and the subsequent transport of material through the disk around the core binary system drives a fast jet-like outflow in the polar directions. While most of the envelope material is lost through a recombination-driven wind from the outer edge of the disk, about $7\,\mathrm{\%}$ of the envelope leaves the system via the magnetically driven outflows. We further explored the potential evolutionary pathways of the post-common-envelope systems given the expected remaining lifetime of the primary core ($2.97\,M_\odot$) until core collapse ($6{\times}10^{4}\,\mathrm{yr}$), most likely forming a neutron star. We find that the interaction of the core binary system with the circumbinary disk increases the likelihood of giving rise to a double-neutron star merger. (abridged)         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.07637-b31b1b.svg)](https://arxiv.org/abs/2410.07637) | **Double-lined Spectroscopic Binaries from the LAMOST Low-Resolution Survey**  |
|| <mark>J. Liu</mark>, B. Zhang, J. Wu, Y.-S. Ting |
|*Appeared on*| *2024-10-11*|
|*Comments*| *29 pages, 23 figures and 5 tables; Accepted for publication in ApJS*|
|**Abstract**|            We report on a data-driven spectral model that we have developed for the identification of double-lined spectroscopic binary stars (SB2s) in the LAMOST low-resolution survey (R$\sim$1800). Employing simultaneous fitting with both single-star and binary-star models, we detected over 4800 SB2 candidates, where both components are detectably contributing to the spectrum, from an initial pool of 2.6 million objects. Tests show that our model favors FGK-type main-sequence binaries with high mass ratio ($q\geq$ 0.7) and large radial velocity separation ($\Delta \rm RV \geq$ 100~km$\,$s$^{-1}$). Almost all of these candidates are positioned above the main sequence in the color-magnitude diagram, indicating their binary nature. Additionally, we utilized various observational data, including spectroscopy, photometry, parallax, and extinction, to determine multiple physical parameters such as the effective temperature, age, metallicity, radial velocity, mass, mass ratio, stellar radius, along with their associated uncertainties for these SB2 candidates. For the 44 candidates with seven or more observational epochs, we provide complete orbital solutions. We make available catalogs containing various stellar parameters for identified SB2 systems.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.08103-b31b1b.svg)](https://arxiv.org/abs/2410.08103) | **A Multi-station Meteor Monitoring (M$^3$) System. II. system upgrade and a pathfinder network**  |
|| Z. Li, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-10-11*|
|*Comments*| *24 pages, 25 figures, Accepted for publication in the Publications of the Astronomical Society of the Pacific (PASP)*|
|**Abstract**|            Meteors are important phenomenon reflecting many properties of interplanetary dust particles. The study of their origin, mass distribution, and orbit evolution all require large data volume, which can only be obtained using large meteor networks. After meteor networks in Europe and America, we present our designs and upgrades of a proposing network in China. The new designs are mainly aimed for facilitating data gathering process. Each of the newly designed meteor stations now can support up to 4 cameras to cover the full sky. Newer version of meteor station software now works as an integral system, which can streamline the process of detecting, measuring and uploading meteors. We have built a meteor data platform to store, process and display the meteor data automatically. The software and data platform are designed to be easy to learn and use, so it can attract more people to join and operate meteor stations. Four stations are installed as the first phase of the network, and during the operation in 10 months, the network detected 8,683 orbits, and we find that half of the orbits can be related to established meteoroid streams. The statistical analysis of sporadic meteoroids shows a bimodal distribution of the velocities, which coincides with previous studies. The distribution of Tisserand parameters, $T_j$, shows the two peaks at $T_j=0$ and 3, indicating the different orbits of parent bodies (isotropic and ecliptic), which are divided by $T_j=2$. The falling trajectory of a meteorite was also predicted using observational data of the network. We are currently expanding the network, and in the future we will carry out detailed analysis of the key parameters of the distribution of the meteoroids.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.08116/./figures/W15_Tiberius_best_fit.png', 'tmp_2410.08116/./figures/W15_Eureka_best_fit.png', 'tmp_2410.08116/./figures/platon_corner_R100_Tiberius+Eureka_lowZ_solutions.png', 'tmp_2410.08116/./figures/platon_corner_R100_Tiberius+Eureka_lowZ_solutions.png', 'tmp_2410.08116/./figures/PLATON_forward+retrieved_models_Tiberius+Eureka_R100_best_fit.png', 'tmp_2410.08116/./figures/WASP-15b_Tiberius_R100_R400_Eureka_comparison_Sep24.png']
copying  tmp_2410.08116/./figures/W15_Tiberius_best_fit.png to _build/html/
copying  tmp_2410.08116/./figures/W15_Eureka_best_fit.png to _build/html/
copying  tmp_2410.08116/./figures/platon_corner_R100_Tiberius+Eureka_lowZ_solutions.png to _build/html/
copying  tmp_2410.08116/./figures/platon_corner_R100_Tiberius+Eureka_lowZ_solutions.png to _build/html/
copying  tmp_2410.08116/./figures/PLATON_forward+retrieved_models_Tiberius+Eureka_R100_best_fit.png to _build/html/
copying  tmp_2410.08116/./figures/WASP-15b_Tiberius_R100_R400_E

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Rjup}{R_{\mathrm{J}}}$
$\newcommand{\Mjup}{M_{\mathrm{J}}}$
$\newcommand{\Rp}{R_{\mathrm{P}}}$
$\newcommand{\Mp}{M_{\mathrm{P}}}$
$\newcommand{\Rs}{R_{\mathrm{*}}}$
$\newcommand{\Ms}{R_{\mathrm{*}}}$
$\newcommand{\Teq}{T_{\mathrm{eq}}}$
$\newcommand{\Teff}{T_{\mathrm{eff}}}$
$\newcommand{◦ee}{^{\circ}}$
$\newcommand{\obliquity}{|\lambda|}$</div>



<div id="title">

# BOWIE-ALIGN: JWST reveals hints of planetesimal accretion and complex sulphur chemistry in the atmosphere of the misaligned hot Jupiter WASP-15b

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.08116-b31b1b.svg)](https://arxiv.org/abs/2410.08116)<mark>Appeared on: 2024-10-11</mark> -  _24 pages, 23 figures, 6 tables. Submitted to MNRAS_

</div>
<div id="authors">

J. Kirk, et al. -- incl., <mark>E.-M. Ahrer</mark>, <mark>P. Molliere</mark>

</div>
<div id="abstract">

**Abstract:** We present a transmission spectrum of the misaligned hot Jupiter WASP-15b from 2.8--5.2 microns observed with JWST's NIRSpec/G395H grating. Our high signal to noise data, which has negligible red noise, reveals significant absorption by $H_2$ O ( $4.2\sigma$ ) and $CO_2$ ( $8.9\sigma$ ). From independent data reduction and atmospheric retrieval approaches, we infer that WASP-15b's atmospheric metallicity is super-solar ( $\gtrsim 15\times$ solar) and its C/O is consistent with solar, that together imply planetesimal accretion. Our GCM simulations for WASP-15b suggest that the C/O we measure at the limb is likely representative of the entire photosphere due to the mostly uniform spatial distribution of $\ch{H2O}$ , $\ch{CO2}$ and CO. We additionally see evidence for absorption by $SO_2$ and absorption at 4.9 $\micron$ , for which the current leading candidate is OCS, albeit with several caveats. If confirmed, this would be the first detection of OCS in an exoplanet atmosphere and point towards complex photochemistry of sulphur-bearing species in the upper atmosphere. These are the first observations from the BOWIE-ALIGN survey which is using JWST's NIRSpec/G395H instrument to compare the atmospheric compositions of aligned/low-obliquity and misaligned/high-obliquity hot Jupiters around F stars above the Kraft break. The goal of our survey is to determine whether the atmospheric composition differs across two populations of planets that have likely undergone different migration histories (disc versus disc-free) as evidenced by their obliquities (aligned versus misaligned).

</div>

<div id="div_fig1">

<img src="tmp_2410.08116/./figures/W15_Tiberius_best_fit.png" alt="Fig7.1" width="50%"/><img src="tmp_2410.08116/./figures/W15_Eureka_best_fit.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Best-fitting models from our \texttt{petitRADTRANS} free chemistry (blue), equilibrium chemistry (orange), and hybrid chemistry (equilibrium chemistry plus free S-bearing species, green) retrievals, fitted to the \texttt{Tiberius}$R=100$ reduction (top panel) and \texttt{Eureka!}$R=100$ reduction (bottom panel). (*fig:prt_best_fit_models*)

</div>
<div id="div_fig2">

<img src="tmp_2410.08116/./figures/platon_corner_R100_Tiberius+Eureka_lowZ_solutions.png" alt="Fig20.1" width="33%"/><img src="tmp_2410.08116/./figures/platon_corner_R100_Tiberius+Eureka_lowZ_solutions.png" alt="Fig20.2" width="33%"/><img src="tmp_2410.08116/./figures/PLATON_forward+retrieved_models_Tiberius+Eureka_R100_best_fit.png" alt="Fig20.3" width="33%"/>

**Figure 20. -** The corner plot and best-fit models from our 1D chemical equilibrium atmosphere retrievals with \texttt{PLATON} run on the $R=100$ spectra, after excluding the unphysically high metallicity solutions. The black contours correspond to the \texttt{Tiberius} retrieval and the red contours to the \texttt{Eureka!} retrieval. The vertical dashed lines indicate the 16th, 50th (median) and 84th percentiles, which are also given in the axes titles. (*fig:PLATON_corner_lowZ*)

</div>
<div id="div_fig3">

<img src="tmp_2410.08116/./figures/WASP-15b_Tiberius_R100_R400_Eureka_comparison_Sep24.png" alt="Fig4" width="100%"/>

**Figure 4. -** WASP-15b's transmission spectrum. Top panel: the comparison between the spectra obtained with \texttt{Tiberius}(black) and \texttt{Eureka!}(red) at $R=100$. The \texttt{Eureka!} spectrum has been offset by -38 ppm to match the median transit depth of \texttt{Tiberius}. Middle panel: The spectrum at $R=100$(black) and $R=400$(gray squares), both obtained with \texttt{Tiberius}. Bottom panel: The spectrum at $R=100$(red) and $R=400$(light red squares), both obtained with \texttt{Eureka!}. (*fig:trans_spec_R100_R400_comparison*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.08116"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

215  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
